Author: Kevin ALBERT  
Created: June 2022  
Friends: [Marco Martins](https://github.com/marco-martins)  

In [1]:
import datetime, time
print ('Last testrun on: ' + datetime.datetime.now().strftime("%d %b %Y"))

Last testrun on: 21 Jul 2022


# Marlowe contract 
_**how to deploy a custom contract**_

## Contents
1. [Objective](#Objective)
1. [Installation](#Installation)
1. [Check](#Check)
1. [Wallets](#Wallets)
1. [Contract](#Contract)
1. [Funding](#Funding)
1. [Tokens](#Tokens)
1. [Deployment](#Deployment)


## Objective

Reference guide that explain and demonstrate what I learned from the Marlowe Pioneer Program  
on how to deploy a custom smart contract on the blockchain using marlowe-cli.  
Python is used together with Linux/GNU shell CLI commands.  
  * cardano-cli  
  * wallet-cli  
  * marlowe-cli  
  


## Installation

### versions

In [2]:
# installed modules in python
conda_version = ! conda -V
print(f"conda       : {conda_version[0].split()[1]}")
pip_version = ! pip -V
print(f"pip         : {pip_version[0].split()[1]}")
python_version = ! python -V
print(f"python      : {python_version[0].split()[1]}")
cardano_version = ! pip list |grep -i cardano
print(f"cardano     : {cardano_version[0].split()[1]}")

conda       : 4.12.0
pip         : 21.2.4
python      : 3.9.12
cardano     : 0.8.2


### modules

In [3]:
# used for loading a webpage
from IPython.display import Javascript

import time         # help calculate POSIX time values
import re           # regex tool
import pandas as pd # tabular data tool
import numpy as np
import json         # json data structure tool

# environment packages
import platform
import psutil
import os

# https://stackabuse.com/executing-shell-commands-with-python/
import subprocess

# pd.describe_option('display')            # show all pandas options, parameters can slow down notebook
pd.set_option('display.max_colwidth', 80)  # default 50, the maximum width in characters of a column
pd.set_option('display.max_columns', 20)   # default 20, the maximum amount of columns in view 
pd.set_option('display.max_rows', 10)      # default 60, the maximum amount of rows in view

### server
You start with **Ubuntu** on a virtual machine in the cloud.

In [4]:
# Virtual Machine environment 
print(f"Cores : {psutil.cpu_count(logical=True)} ({psutil.cpu_freq().current/1000:.0f}GHz)")
print(f"Memory: {psutil.virtual_memory().total/(1024**3):.2f} GB ({psutil.virtual_memory().percent}%)")
print(f"Swap  : {os.path.getsize('/swapfile')/(1024**3):.0f} GB")
disk_size = psutil.disk_usage(psutil.disk_partitions()[0].mountpoint).total
disk_used = psutil.disk_usage(psutil.disk_partitions()[0].mountpoint).percent
disk_fs   = psutil.disk_partitions()[0].fstype 
print(f"Disk  : {disk_size/(1024**3):.0f} GB ({disk_used}% {disk_fs})")
print(f"System: {platform.uname().version.split('~')[1].split()[0]}")

Cores : 8 (3GHz)
Memory: 62.81 GB (12.9%)
Swap  : 8 GB
Disk  : 242 GB (48.1% ext4)
System: 20.04.1-Ubuntu


### install
Connect to your VM using Putty.  
**Update** your installation.  
Install **curl** and **rsync**.  
```sh
sudo sh -c 'apt update && apt install curl rsync'
```

**Install nixos** from the official installation script.  
Nixos is a form of virtual environment like docker with cryptographic hashed package manager Nix to eliminate dependency hell.
```sh
sh <(curl -L https://nixos.org/nix/install) --daemon
```

Add content to **nix.conf** and **restart** the nix-daemon.  
These lines allow for a fast build of nixos, downloading trusted binaries instead of compiling from scratch.  
```sh
sudo sh -c "echo 'substituters = https://hydra.iohk.io https://iohk.cachix.org https://cache.nixos.org/' >> /etc/nix/nix.conf"

sudo sh -c "echo 'trusted-public-keys = hydra.iohk.io:f/Ea+s+dFdN+3Y/G+FDgSq+a5NEWhJGzdjvKNGv0/EQ= iohk.cachix.org-1:DpRUyj7h7V830dp/i6Nti+NEO2/nhblbov/8MW7Rqoo= cache.nixos.org-1:6NCHdD59X431o0gWypbMrAURkbJ16ZPMQFGspcDShjY=' >> /etc/nix/nix.conf"

sudo cat /etc/nix/nix.conf

sudo sh -c 'echo "trusted-users = $0" >> /etc/nix/nix.conf' `whoami`
```

In [5]:
# here is the content of the config file
!cat /etc/nix/nix.conf


build-users-group = nixbld
substituters = https://hydra.iohk.io https://iohk.cachix.org https://cache.nixos.org/
trusted-public-keys = hydra.iohk.io:f/Ea+s+dFdN+3Y/G+FDgSq+a5NEWhJGzdjvKNGv0/EQ= iohk.cachix.org-1:DpRUyj7h7V830dp/i6Nti+NEO2/nhblbov/8MW7Rqoo= cache.nixos.org-1:6NCHdD59X431o0gWypbMrAURkbJ16ZPMQFGspcDShjY=
trusted-users = ubuntu


Restart Nix  
```sh
sudo systemctl restart nix-daemon.service
```

Download **git repo**sitory and set to marlowe-pioneers branch (special testnet).  
```sh
git clone https://github.com/input-output-hk/marlowe-cardano.git
cd marlowe-cardano/
git checkout marlowe-pioneers
```

start up **nix-shell**  
first time it will install and update itself  
```sh
nix-shell
```

**start** the virtual environment with CLI tools and blockchain node.  
Wait, let it synchronise... once started it will open a **tmux** session.  
tmux, where multiple terminal session windows are available by default.  

```sh
start-marlowe-run
```  

Here are some commands to use tmux:  

    PANE commands
    ctrl+b ->    to move to the right pane, up, down and left arrow also possible
    ctrl+b z     enlarge or zoom to one pane
    ctrl+b "     for split horizontal
    ctrl+b %     for split vertikal
    ctrl+b d     to kill a screen pane
    
    WINDOW commands
    ctrl+b c     create new window
    ctrl+b p     previous window
    ctrl+b n     next window
    ctrl+b &     kill a window
    
    SESSION commands
    tmux list-sessions    view a list of all available sessions
    tmux ls               (short version)
    tmux attach           previous last used tmux session
    tmux a                (short version)
    tmux attach -t 1      open tmux session id 1

### custom helper functions

In [514]:
def query_df(address, magic='1567', cardanoCLI="/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli"):
    '''
    query_df(address, magic, cardanoCLI)
    
    address : (str) [required] is the wallet address you want to check for last transaction state
    magic : (str) [optional] is the key used to identify the mainnet, testnet, other networks
    cardanoCLI : (str) [optional] the binary file of the cardano-cli on your system
    '''

    def hex_to_ascii(s):
        # small function to decode hex strings to ascii (translation)
        try:
            return bytearray.fromhex(s).decode()
        except ValueError:
            return None

    # shell command, execute and store result in a temporary json file
    cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", magic, "--address", address, "--out-file", 'address.json']
    result = subprocess.run(cmd, capture_output=True, text=True)
    # load this json file into a dataframe
    df = pd.read_json('address.json', orient='index').reset_index()

    # test if empty query, then return empty line with address
    if len(df.columns) < 2:
        # close off
        return pd.DataFrame({'address':[address], 'TxHash':np.nan, 'TxIx':0, 'lovelace':0, 'TokenPolicyId':np.nan, 'TokenNameHex':np.nan, 'TokenNameAscii':np.nan, 'TokenAmount':0, 'datumhash':np.nan})

    # replace feature names with interpretable naming (others are left as-is) 
    df = df.rename(columns={'index': 'TxHash',
                            'datumhash': 'datumhash',
                            'address': 'address',
                            'value': 'value'})
    # split the # value into a seperate column and remove the original
    df = df.join(df['TxHash'].str.split('#', regex=False, expand=True), how='left')
    df = df.drop(['TxHash'], axis=1)
    # replace feature names with interpretable naming (others are left as-is) 
    df = df.rename(columns={0:'TxHash', 1: 'TxIx'})
    # normalize the value column ex:{lovelace:3000000} into 2 new columns and remove original column
    df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
    # change data type from float to integer
    df['lovelace'] = df['lovelace'].fillna(0)
    df['TxIx'] = df['TxIx'].fillna(0)
    df['lovelace'] = df['lovelace'].astype('int')
    df['TxIx'] = df['TxIx'].astype('int')
    # drop all columns with all values NA
    df = df.dropna(axis=1, how='all')
    
    # if the last column name word size is 56 characters wide:
    # try if this is needed, if no data is present then skip
    try:
        # unselect all known columns, keep last ones column name
        cn = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace'])]].iloc[:, -1].name
        # check if this name is 64 bytes long ? 
        if len(cn) == 56:
            # make a copy into a new column name
            df['TokenPolicyId'] = df[cn]
            # write the values 'column name' only on True state locations
            df['TokenPolicyId'] = df['TokenPolicyId'].notna().replace({True: cn, False: np.nan})
            
            # normalize the value column ex:{}
            df = pd.concat([df.drop([cn], axis=1), df[cn].apply(pd.Series)], axis=1)
            # drop all columns with all values NA
            df = df.dropna(axis=1, how='all')
            
            # build a secondary dataframe to be concatenated at the end with the hextokenid and tokenamount
            # exclude all known columns
            two = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]]
            # stack the column name and values into one dataframe and reset the index
            tmp = two.stack().apply(pd.Series).reset_index()
            # set the index to this column values, later for merging with df
            tmp = tmp.set_index('level_0')
            # rename these 2 columns
            tmp.columns = ['TokenNameHex', 'TokenAmount']
            # merge
            df = pd.concat([df, tmp], axis=1)
            # set tokenamount to integer dtype
            df['TokenAmount'] = df['TokenAmount'].fillna(0)
            df['TokenAmount'] = df['TokenAmount'].astype('int')
            # remove the bogus columns, clean up
            df = df.drop(tmp['TokenNameHex'], axis=1)
            # add a column with the ascii transation
            df['TokenNameAscii'] = df['TokenNameHex'].astype('str').apply(hex_to_ascii)
            # add empty column data so we can combine all dataframes into one later
            df['datumhash'] = np.nan
            # structure in chronologic order the column names
            df = df[['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId', 'TokenNameHex', 'TokenNameAscii', 'TokenAmount', 'datumhash']]
            return df
            
    # in the case of no data present add these extra empty columns
    # in fact this will never occur, below code... 
    except:
        df['datumhash'] = np.nan
        df['TokenPolicyId'] = np.nan
        df['TokenNameHex'] = np.nan
        df['TokenAmount'] = 0
        return df

    # add empty column data so we can combine all dataframes into one later
    df['TokenPolicyId'] = np.nan
    df['TokenNameHex'] = np.nan
    df['TokenNameAscii'] = np.nan
    df['TokenAmount'] = 0
    # structure in chronologic order the column names
    df = df[['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId', 'TokenNameHex', 'TokenNameAscii', 'TokenAmount', 'datumhash']]
    return df

## Check

### find binaries
Here we search for the binaries installed in nix-shell.  

  * **cardano-cli** used for generating keys, constructing transactions, creating certificates on the cardano node running in the background
  * **marlowe-cli** used for serialising Marlowe contracts to validators, datums, redeemers and also computes hashes and addresses with cardano-cli
  * **cardano-wallet** used for sending and receiving payments, list, create, update, or delete wallets  
  
  

In [7]:
!sudo find /nix/store -wholename '*/bin/cardano-cli'
!sudo find /nix/store -wholename '*/bin/marlowe-cli'
!sudo find /nix/store -wholename '*/bin/cardano-wallet'

/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli
/nix/store/9ijr3m1g4rr487q8gz35h3d27bfrxkyw-marlowe-cli-exe-marlowe-cli-0.0.4.4/bin/marlowe-cli
/nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet


### save binary path

In [8]:
cardanoCLI = "/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli"
marloweCLI = "/nix/store/9ijr3m1g4rr487q8gz35h3d27bfrxkyw-marlowe-cli-exe-marlowe-cli-0.0.4.4/bin/marlowe-cli"
cardanoWALLET = "/nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet"

### check versions

latest version available:
  * [cardano-cli](https://github.com/input-output-hk/cardano-node/releases/latest)
  * [marlowe-cli](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli)
  * [cardano-wallet](https://github.com/input-output-hk/cardano-wallet/releases/latest)
  
below we show the current version used when nix-shell installed

In [9]:
!/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli --version

cardano-cli 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [10]:
!/nix/store/9ijr3m1g4rr487q8gz35h3d27bfrxkyw-marlowe-cli-exe-marlowe-cli-0.0.4.4/bin/marlowe-cli --version

marlowe-cli 0.0.4.4


In [11]:
!/nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet version

v2022-01-18 (git revision: 0000000000000000000000000000000000000000)


### node.socket
The cardano-node uses IPC (Inter-Process-Communication) for communicating with other Cardano components.  
Components like cardano-cli, cardano-wallet, marlowe-cli and cardano-db-sync.  
So, basically this file helps connect your CLI instance to the active cardano-node currently syncing the blockchain.  

In [12]:
# detect the location of the cardano node "socket"
!sudo find /tmp -name '*node.socket'

/tmp/node.socket


In [13]:
!date
!ls -al /tmp/node.socket

Thu Jul 21 17:35:40 CEST 2022
srwxr-xr-x 1 ubuntu ubuntu 0 Jun 28 17:35 /tmp/node.socket


In order to run cardano-cli in nix-shell bash commands:  

```shell
sudo vi /etc/environment
```
add the following
```
CARDANO_NODE_SOCKET_PATH="/tmp/node.socket"
```

In [14]:
# this is only needed for cardano-cli, not marlowe-cli
os.environ['CARDANO_NODE_SOCKET_PATH']="/tmp/node.socket"
# check the result, call the environment variable from within jupyter notebook
!echo $CARDANO_NODE_SOCKET_PATH

/tmp/node.socket


In [15]:
# list all environment variables defined in the notebook
os.environ

environ{'SHELL': '/bin/bash',
        'CONDA_EXE': '/anaconda/bin/conda',
        '_CE_M': '',
        'JUPYTERHUB_API_TOKEN': '913ad996b9fe49b39ea72e40fa9eb805',
        'JUPYTERHUB_BASE_URL': '/',
        'PWD': '/home/ubuntu/notebooks/azuremachinelearning/code/notebooks',
        'CONDA_PREFIX': '/anaconda/envs/py39_cardano',
        'JUPYTERHUB_SERVER_NAME': '',
        'HOME': '/home/ubuntu',
        'JPY_API_TOKEN': '913ad996b9fe49b39ea72e40fa9eb805',
        'CONDA_PROMPT_MODIFIER': '(py39_cardano) ',
        'JUPYTERHUB_SERVICE_PREFIX': '/user/ubuntu/',
        'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/ubuntu/oauth_callback',
        'JPY_PARENT_PID': '94408',
        '_CE_CONDA': '',
        'USER': 'ubuntu',
        'CONDA_SHLVL': '1',
        'SHLVL': '0',
        'JULIA_DEPOT_PATH': '/opt/julia/latest/packages/',
        'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-ubuntu',
        'JUPYTERHUB_HOST': '',
        'COND

## Wallets

### sync

Be patient and wait for the cardano-node in nix-shell to download and **synchronize the blockchain history.**  
This can take many hours.  

After sync, the cardano-wallet can be used either through  
HTTP Application Programming Interface (**API**) and command-line interface (**CLI**)  

  * [Python Cardano module](https://github.com/emesik/cardano-python)

**[python module for cardano-wallet working on mainnet 'notebook'](./howto_cardano-wallet.ipynb)**

## wallet

### generate

mnemonic seed phrase:  

  * 12 words (Byron legacy wallet)
  * 15 words (Incentivised Testnet Rewards wallet)
  * **24 words** (Shelley wallet)
  * 27 words (Byron legacy paper wallet)
  
generate new Shelley wallet  
```sh
cardano-wallet recovery-phrase generate --size 24
```
generated example output:
```sh
movie use luxury dumb kingdom garden mammal strong laundry weasel legal mesh movie old cancel paddle color pluck slight wheel mouse front slice pupil
```

[**how secure are seed phrases**](https://www.reddit.com/r/CryptoCurrency/comments/w0bv37/how_secure_are_seed_phrases)  

Convert the seed phrase into a root private key 'wallet1.prv'.  
```sh
cat wallet1.seed | cardano-wallet key from-recovery-phrase Shelley | cardano-wallet key child 1852H/1815H/0H/0/0 > wallet1.prv
```

Generate the *.**skey**, *.**vkey** and *.**address** files (using *.prv)
```sh
cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file wallet1.prv --out-file wallet1.skey
cardano-cli key verification-key --signing-key-file wallet1.skey --verification-key-file wallet1.vkey
cardano-cli address build --testnet-magic 1567 --payment-verification-key-file wallet1.vkey > wallet1.addr
```

**Question:** I don't understand why the size of the public address is smaller than a normal cardano-wallet address ?  
**Question:** I don't understand why we need a pkey, skey, vkey and what they mean ?  
**Question:** I don't understand why we need these files for marlowe-cli transactions and how do we use a light wallet, web wallet instead of these skey, vkey ?  
**Question:** what is the purpose of the vkey (=verification key) ?  
**Question:** what is the purpose of the skey (=signing key) ?  
**Question:** is the private key *.prv somehow the translation from the seed phrase to a different form, each word representing 4 digit number ?  

READ THIS: https://developers.cardano.org/docs/integrate-cardano/creating-wallet-faucet#creating-a-wallet

#### wallet 1

In [16]:
# generate 24 word key phrase
cmd = [cardanoWALLET, "recovery-phrase", "generate", "--size", str(24)]  # cardano-wallet CLI command to generate NEW mneumic seed phrase with 24 words
result = subprocess.run(cmd, capture_output=True, text=True)             # run the command
print("wallet 1 seed phrase:\n\n", result.stdout)                        # demonstrate the result
open('wallet1.seed', 'w').write(result.stdout)                           # save the result to a file 'wallet1.seed'

wallet 1 seed phrase:

 motion mechanic crucial sentence focus merge correct festival impact wife twelve loyal catch seat leg outer bid penalty left valley trial chimney upgrade elite



160

In [17]:
# generate the address, skey and vkey files
!cat wallet1.seed | /nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet key from-recovery-phrase Shelley | /nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet key child 1852H/1815H/0H/0/0 > wallet1.prv
!/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file wallet1.prv --out-file wallet1.skey
!/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli key verification-key --signing-key-file wallet1.skey --verification-key-file wallet1.vkey
!/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli address build --testnet-magic 1567 --payment-verification-key-file wallet1.vkey > wallet1.address

In [18]:
!cat wallet1.prv

addr_xsk1wptvk7ck44jjsng8nxd0ukwht2rq9qfwse8ugfsch8ytjqkfc3xsc7s7pegvzq084l5v5dfy0g49e238rzqzjt9lgshrqcpwgj4uf6fcqfa7gqt6cu0dg5zmhhqt4995zg3uvk8cldv2m0fvsccj5l726ursatrr

In [19]:
!cat wallet1.skey

{
    "type": "PaymentExtendedSigningKeyShelley_ed25519_bip32",
    "description": "",
    "cborHex": "58807056cb7b16ad65284d07999afe59d75a8602812e864fc42618b9c8b902c9c44d0c7a1e0e50c101e7afe8ca35247a2a5caa271880292cbf442e30602e44abc4e97060d53791200399a29871511d5f35bd8fd06cfb177c54fd4886c4edffa5c78938027be4017ac71ed4505bbdc0ba94b41223c658f8fb58adbd2c86312a7fcad7"
}


In [20]:
!cat wallet1.vkey

{
    "type": "PaymentExtendedVerificationKeyShelley_ed25519_bip32",
    "description": "",
    "cborHex": "58407060d53791200399a29871511d5f35bd8fd06cfb177c54fd4886c4edffa5c78938027be4017ac71ed4505bbdc0ba94b41223c658f8fb58adbd2c86312a7fcad7"
}


In [21]:
!cat wallet1.address

addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t

#### wallet 2

In [22]:
# generate 24 word key phrase
cmd = [cardanoWALLET, "recovery-phrase", "generate", "--size", "24"]
result = subprocess.run(cmd, capture_output=True, text=True)
print("wallet 2 seed phrase:\n\n", result.stdout)
open('wallet2.seed', 'w').write(result.stdout)

wallet 2 seed phrase:

 produce rural buffalo easily wagon burst scout garage piano pass faith speed athlete witness popular north umbrella burst music draft lyrics camp glue tunnel



158

In [23]:
!cat wallet2.seed | /nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet key from-recovery-phrase Shelley | /nix/store/9n5z901h18k7ijjj8axvyv4w2z0zgqcv-cardano-wallet-exe-cardano-wallet-2022.1.18/bin/cardano-wallet key child 1852H/1815H/0H/0/0 > wallet2.prv
!/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file wallet2.prv --out-file wallet2.skey
!/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli key verification-key --signing-key-file wallet2.skey --verification-key-file wallet2.vkey
!/nix/store/vaqzj4d6c1fdmqmm78y3g4n10pyw5mmc-cardano-cli-exe-cardano-cli-1.34.1/bin/cardano-cli address build --testnet-magic 1567 --payment-verification-key-file wallet2.vkey > wallet2.address

In [28]:
!cat wallet2.address

addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6

In [24]:
# list all the wallet files generated
!date
!ls -al wallet*

Thu Jul 21 17:36:18 CEST 2022
-rw-rw-r-- 1 ubuntu ubuntu  63 Jul 21 17:36 wallet1.address
-rw-rw-r-- 1 ubuntu ubuntu 169 Jul 21 17:35 wallet1.prv
-rw-rw-r-- 1 ubuntu ubuntu 160 Jul 21 17:35 wallet1.seed
-rw------- 1 ubuntu ubuntu 367 Jul 21 17:36 wallet1.skey
-rw------- 1 ubuntu ubuntu 244 Jul 21 17:36 wallet1.vkey
-rw-rw-r-- 1 ubuntu ubuntu  63 Jul 21 17:36 wallet2.address
-rw-rw-r-- 1 ubuntu ubuntu 169 Jul 21 17:36 wallet2.prv
-rw-rw-r-- 1 ubuntu ubuntu 158 Jul 21 17:36 wallet2.seed
-rw------- 1 ubuntu ubuntu 367 Jul 21 17:36 wallet2.skey
-rw------- 1 ubuntu ubuntu 244 Jul 21 17:36 wallet2.vkey


## Contract

### design
We can develop a smart contract in different ways, javascript, haskell, marlowe and more in the future.  
This example was written by Marco in javascript (.js) and can be [**found here.**](https://gist.github.com/marco-martins/670832702aaa2c9258cfecef5a1b7fa3)  
The idea here is to have 2 wallet users each sending a certain amount in ADA  
and then each receives what the other person sended, called a swap.  
Consult the [**marlowe playground**](https://marlowe-playground-marlowe-pioneers.plutus.aws.iohkdev.io/#/) for editing and simulating how this works.  
![marlowe playground javascript code replacement](../../image/marlowe_playground_swap_javascript.png)

Alternatively you can use the following **Marlowe** code (also found after compilation).  

```sh
When
    [Case
        (Deposit
            (Role "Seller")
            (Role "Seller")
            (Token "" "")
            (ConstantParam "Seller Amount")
        )
        (When
            [Case
                (Deposit
                    (Role "Buyer")
                    (Role "Buyer")
                    (Token "" "")
                    (ConstantParam "Buyer Amount")
                )
                (Pay
                    (Role "Seller")
                    (Party (Role "Buyer"))
                    (Token "" "")
                    (ConstantParam "Seller Amount")
                    (Pay
                        (Role "Buyer")
                        (Party (Role "Seller"))
                        (Token "" "")
                        (ConstantParam "Buyer Amount")
                        Close 
                    )
                )]
            (TimeParam "Deadline")
            Close 
        )]
    (TimeParam "Deadline")
    Close 
```

You can then also view as blocks and code visually using **blockly**.  
  
![marlowe playground blockly code](../../image/marlowe_playground_swap_blockly.png)  


**You can export the script to ".json"**  
We stored this script and it's content looks like this

In [25]:
!cat ../../data/contracts/swap_contract.json

{"when":[{"then":{"when":[{"then":{"token":{"token_name":"","currency_symbol":""},"to":{"party":{"role_token":"Buyer"}},"then":{"token":{"token_name":"","currency_symbol":""},"to":{"party":{"role_token":"Seller"}},"then":"close","pay":0,"from_account":{"role_token":"Buyer"}},"pay":0,"from_account":{"role_token":"Seller"}},"case":{"party":{"role_token":"Buyer"},"of_token":{"token_name":"","currency_symbol":""},"into_account":{"role_token":"Buyer"},"deposits":0}}],"timeout_continuation":"close","timeout":1658230520814},"case":{"party":{"role_token":"Seller"},"of_token":{"token_name":"","currency_symbol":""},"into_account":{"role_token":"Seller"},"deposits":0}}],"timeout_continuation":"close","timeout":1658230520814}

In [26]:
with open('../../data/contracts/swap_contract.json', 'r') as file:
    parsed = json.load(file)
    print(json.dumps(parsed, indent=2))

{
  "when": [
    {
      "then": {
        "when": [
          {
            "then": {
              "token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "to": {
                "party": {
                  "role_token": "Buyer"
                }
              },
              "then": {
                "token": {
                  "token_name": "",
                  "currency_symbol": ""
                },
                "to": {
                  "party": {
                    "role_token": "Seller"
                  }
                },
                "then": "close",
                "pay": 0,
                "from_account": {
                  "role_token": "Buyer"
                }
              },
              "pay": 0,
              "from_account": {
                "role_token": "Seller"
              }
            },
            "case": {
              "party": {
                "role_token": "Buyer"
              

### parameters
The **parties agree beforehand to define certain parameter values** to be used for the contract script.  
Notice that the values can only be **integers** to be precise when used on chain.  
This is to remove machine independable float value discrepancies.  
So, even time is translated to an integer representation for this reason,  
so is the value of ADA currently having 5 decimal points called lovelaces.  
The rest is often represented as **string** values.  

In [30]:
ADA                   = 1000000         # 1 ADA = 1000000 lovelace
MILLISECOND           = 1000
NOW                   = round(time.time())*1000
HOUR                  = 60*60*MILLISECOND
PAYMENT_DEADLINE      = NOW+10*HOUR     # The payment deadline, ten hours from now
MINIMUM_ADA           = 3*ADA           # The minimum lovelace to be included with native token output

PARTY1_TOKENASSETNAME = 'Party1'        # this is 'Buyer', 'wallet 1'
PARTY2_TOKENASSETNAME = 'Party2'        # this is 'Seller', 'wallet 2'
PARTY1_AMOUNT         = 200*ADA         # 200000000, means we will hard-code this swap amount in the script
PARTY2_AMOUNT         = 100*ADA         # 100000000, means we will hard-code this swap amount in the script
PARTY1_ADDRESS        = open('wallet1.address', 'r').read() # string value 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
PARTY2_ADDRESS        = open('wallet2.address', 'r').read() # string value 'addr_test1vrfhx0w0fy9aad9kskezy5znpmey0h335gcjen2pp4gjj2s2ax92x'
PARTY1_SKEYFILE       = 'wallet1.skey'  # string value pointing to path/file
PARTY2_SKEYFILE       = 'wallet2.skey'
PARTY1_VKEYFILE       = 'wallet1.vkey'
PARTY2_VKEYFILE       = 'wallet2.vkey'

MAGIC                 = '1567'          # marlowe-pioneers testnet ID (provided by developers)
SOCKET                = '/tmp/node.socket'

# the values below will be created further down the road, 
# it can be interesting to list them here for your understanding
TOKEN_POLICY_ID       = ''  # minted token Policy ID, '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43'
TX1_IN                = ''  # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9'
TX1_PARTY1_ADA        = ''  # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#0'
TX1_PARTY1_TOKENS     = ''  # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#1'
TX1_PARTY2_ADA        = ''  #
TX1_PARTY2_TOKENS     = ''  #
TX2_IN                = ''  # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373'
CONTRACT_ADDRESS      = ''  # 'addr_test1wp544j8t4m3a32g6apad3lpjjfj09j948y8m4nthc5e9rgg5uxu2z'
REDEEM_ADDRESS        = ''  # 'addr_test1wr0970a9k82v6pcw68x2yzwm737zyepg9y8ar8jxsxstf6gzpsmyv'

### script
The goal here is to hard-code substitute parameters in the json script,  
replacing with real values the token name, amounts, timeout values, and so on...  

In [31]:
# currency_symbol = PolicyID of the minted tokens (ex: "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43")
PARTY1_TOKENID   = ""  # empty (default to ADA)
PARTY2_TOKENID   = ""  # empty (default to ADA)

# token_name = currency name of the minted tokens (ex: "CHOC")
PARTY1_TOKENNAME = ""  # empty (default to ADA)
PARTY2_TOKENNAME = ""  # empty (default to ADA)

TO BE replaced below later these token names !! then it needs to be tested...  
Maybe we can later add the minting of tokens and try to swap these as an example.  

In [32]:
result = subprocess.run("""cat <<EOF > tx-1.contract
{
  "when": [
    {
      "then": {
        "when": [
          {
            "then": {
              "token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "to": {
                "party": {
                  "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
                }
              },
              "then": {
                "token": {
                  "token_name": "",
                  "currency_symbol": ""
                },
                "to": {
                  "party": {
                    "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
                  }
                },
                "then": "close",
                "pay": """+str(PARTY2_AMOUNT)+""",
                "from_account": {
                  "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
                }
              },
              "pay": """+str(PARTY1_AMOUNT)+""",
              "from_account": {
                "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
              }
            },
            "case": {
              "party": {
                "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
              },
              "of_token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "into_account": {
                "role_token": '"""+PARTY2_TOKENASSETNAME+"""'
              },
              "deposits": """+str(PARTY2_AMOUNT)+"""
            }
          }
        ],
        "timeout_continuation": "close",
        "timeout": """+str(PAYMENT_DEADLINE)+"""
      },
      "case": {
        "party": {
          "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
        },
        "of_token": {
          "token_name": "",
          "currency_symbol": ""
        },
        "into_account": {
          "role_token": '"""+PARTY1_TOKENASSETNAME+"""'
        },
        "deposits": """+str(PARTY1_AMOUNT)+"""
      }
    }
  ],
  "timeout_continuation": "close",
  "timeout": """+str(PAYMENT_DEADLINE)+"""
}
EOF
""", shell=True)

In [33]:
!cat tx-1.contract

{
  "when": [
    {
      "then": {
        "when": [
          {
            "then": {
              "token": {
                "token_name": "",
                "currency_symbol": ""
              },
              "to": {
                "party": {
                  "role_token": 'Party2'
                }
              },
              "then": {
                "token": {
                  "token_name": "",
                  "currency_symbol": ""
                },
                "to": {
                  "party": {
                    "role_token": 'Party1'
                  }
                },
                "then": "close",
                "pay": 100000000,
                "from_account": {
                  "role_token": 'Party2'
                }
              },
              "pay": 200000000,
              "from_account": {
                "role_token": 'Party1'
              }
            },
            "case": {
              "party": {
                "role_token": 'Par

Question: I don't understand yet why minTime always is 1,  
I believe it has to do with the start of the existence of the blockchain.  
One can believe we need to set the startdate NOW in there.  
Question: why is there no secondary account party 2 in here, what is this state file for ?  
is this to do the initial startup of the contract somehow ?  
Question: Why do we need 2 or 3 ADA here ?  
I believe this is to counter bursting of DoS attacks in contracts initialisations...  

In [34]:
result = subprocess.run("""cat <<EOF > tx-1.state
{
  "accounts": [
    [[{ "role_token": '"""+PARTY1_TOKENASSETNAME+"""'}, { "currency_symbol": "", "token_name": "" }], """+str(MINIMUM_ADA)+"""]
  ],
  "choices": [],
  "boundValues": [],
  "minTime": """+str(1)+"""
}
""", shell=True)

In [35]:
!cat tx-1.state

{
  "accounts": [
    [[{ "role_token": 'Party1'}, { "currency_symbol": "", "token_name": "" }], 3000000]
  ],
  "choices": [],
  "boundValues": [],
  "minTime": 1
}


### template (optional)
Using a template, one can construct with marlowe-cli the *.contract and *.state files  
Today these templates are available:  
  * escrow
  * simple
  * swap
  * zcb (zero-coupon bond)
  * coveredCall

In [36]:
# minted tokenID.tokenNAME == currency_symbol.token_name
PARTY1_TOKENID   = "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43"
PARTY1_TOKENNAME = "BERRY"
PARTY2_TOKENID   = "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43"
PARTY2_TOKENNAME = "CHOC"

```text
marlowe-cli template swap:

  --minimum-ada INTEGER             Lovelace that the first party contributes to the initial state
  --a-party PARTY                   The first party
  --a-token TOKEN                   The first party's token
  --a-amount INTEGER                The amount of the first party's token
  --a-timeout POSIX_TIME            The timeout for the first party's deposit, in POSIX milliseconds
  --b-party PARTY                   The second party
  --b-token TOKEN                   The second party's token
  --b-amount INTEGER                The amount of the second party's token
  --b-timeout POSIX_TIME            The timeout for the second party's deposit, in POSIX milliseconds
  --out-contract-file CONTRACT_FILE JSON output file for the contract
  --out-state-file STATE_FILE       JSON output file for the contract's state

```

In [37]:
# this example seem not to work for swapping ADA, don't understand the syntax (we will not use these files)
cmd = [marloweCLI, "template", "swap", 
       "--minimum-ada", str(MINIMUM_ADA),                # '3000000'
       "--a-party", "Role="+PARTY1_TOKENASSETNAME,       # 'Role=Party1'
       "--a-token", PARTY1_TOKENID+"."+PARTY1_TOKENNAME, # '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.BERRY'
       "--a-amount", str(PARTY1_AMOUNT),                 # '200000000'
       "--a-timeout", str(PAYMENT_DEADLINE),             # '1658275769000'
       "--b-party", "Role="+PARTY2_TOKENASSETNAME,       # 'Role=Party2'
       "--b-token", PARTY2_TOKENID+"."+PARTY2_TOKENNAME, # '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.CHOC'
       "--b-amount", str(PARTY2_AMOUNT),                 # '100000000'
       "--b-timeout", str(PAYMENT_DEADLINE),             # '1658275769000'
       "--out-contract-file", "swap_template.contract",
       "--out-state-file", "swap_template.state"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [38]:
!cat swap_template.contract

{
    "timeout": 1658454012000,
    "when": [
        {
            "then": {
                "timeout": 1658454012000,
                "when": [
                    {
                        "then": {
                            "then": {
                                "then": "close",
                                "to": {
                                    "party": {
                                        "role_token": "Party1"
                                    }
                                },
                                "from_account": {
                                    "role_token": "Party2"
                                },
                                "pay": 100000000,
                                "token": {
                                    "currency_symbol": "0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43",
                                    "token_name": "CHOC"
                                }
                            },
              

In [39]:
!cat swap_template.state

{
    "choices": [],
    "boundValues": [],
    "accounts": [
        [
            [
                {
                    "role_token": "Party1"
                },
                {
                    "currency_symbol": "",
                    "token_name": ""
                }
            ],
            3000000
        ]
    ],
    "minTime": 1
}

## Transactions

In [1]:
from IPython.display import IFrame

# visual of the steps the contract will go through
url = "https://miro.com/app/board/uXjVP3tf9_E=/?moveToWidget=3458764542137977013&cot=14"
IFrame(url, width="800", height="600")

## Funding

### faucet
Request from the testnet to send **500 tADA** (test ADA) into wallet1 and wallet2.  

```text
marlowe-cli util faucet:

  --testnet-magic INTEGER   Network magic
  --socket-path SOCKET_FILE Location of the cardano-node socket file
  --lovelace LOVELACE       The lovelace to send to each address
  --out-file FILE           Output file for transaction body
  --submit SECONDS          Also submit the transaction, and wait for confirmation
  ADDRESS                   The addresses to receive the funds
```

In [40]:
# fund both address with for example 500 tADA - check in Daedalus
cmd = [marloweCLI, "util", "faucet",
       "--testnet-magic", MAGIC,       # '1567' = marlowe-pioneer-testnet
       "--lovelace", str(500 * ADA),   # '500000000'
       "--out-file", "/dev/null",      # discard the send data
       "--submit", str(600),           # execution timeout limit 600 seconds, should finish in 20 sec = 1 block
       PARTY1_ADDRESS, PARTY2_ADDRESS] # list all wallet addresses to send tADA towards, 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33"




In [518]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN


## Tokens

### minting
We chose to mint 2 tokens using wallet 1, one for each role.  
We pay a fee for minting these tokens from this wallet.  
One can argue to mint on any wallet and distribute.  

```text
marlowe-cli util mint:

  --testnet-magic INTEGER        Network magic
  --socket-path SOCKET_FILE      Location of the cardano-node socket file
  --required-signer SIGNING_FILE File containing a required signing key
  --metadata-file JSON_FILE      The CIP-25 metadata, with keys for each token name
  --count INTEGER                The number of each token to mint
  --expires SLOT_NO              The slot number after which miniting is no longer possible
  --lovelace LOVELACE            The lovelace to send with each bundle of tokens
  --change-address ADDRESS       Address to receive ADA in excess of fee
  --out-file FILE                Output file for transaction body
  --submit SECONDS               Also submit the transaction, and wait for confirmation
  TOKEN_NAME                     The name of the token
```

In [41]:
cmd = [marloweCLI, "util", "mint", 
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
       "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--out-file", "mint.raw",
       "--submit", str(600),
       PARTY1_TOKENASSETNAME, PARTY2_TOKENASSETNAME]   # 'Party1', 'Party2', do not use quotes "PARTY1_TOKENASSETNAME"
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

PolicyID "6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508"




In [42]:
# The policy ID will be used for the Marlowe roles currency.  
# Save this minted token Policy ID variable.  
TOKEN_POLICY_ID = result.stdout.split('"')[1]  
TOKEN_POLICY_ID

'6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508'

In [519]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN


In [44]:
TX0_IN = "e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea"

### distribute
send the two tokens created with wallet 1 out to each participant in one transaction.  
wallet 1 must receive 'Party1' token and some minimum currency (3 tADA),  
wallet 2 must receive 'Party2' token and some minimum currency (3 tADA).  

Alternatively, one can distribute the tokens using any wallet software like Daedalus as well.

```text
marlowe-cli transaction simple:

  --testnet-magic INTEGER         Network magic
  --socket-path SOCKET_FILE       Location of the cardano-node socket file
  --required-signer SIGNING_FILE  File containing a required signing key
  --tx-in TXID#TXIX               Transaction input in TxId#TxIx format
  --tx-out ADDRESS+VALUE          Transaction output in ADDRESS+VALUE format
  --change-address                ADDRESS Address to receive ADA in excess of fee
  --metadata-file METADATA_FILE   JSON file containing metadata
  --out-file FILE                 Output file for transaction body
  --submit SECONDS                Also submit the transaction, and wait for confirmation
  --print-stats                   Print statistics
  --script-invalid                Assert that the transaction is invalid
```

In [337]:
# distribute the tokens that were minted on wallet1 around all wallets.

cmd = [marloweCLI, "transaction", "simple",  # Build a non-Marlowe transaction
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
       "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--tx-in", TX0_IN+"#0",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#0'
       "--tx-in", TX0_IN+"#1",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#1'
       "--tx-in", TX0_IN+"#2",               # 'ddba65eeff7e4b3e0a85001a1316be04e60b739f239c05e4c0d206c7694b8d06#2'
       "--tx-out", PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,
       # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party1'
       "--tx-out", PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,
       # 'addr_test1vrfhx0w0fy9aad9kskezy5znpmey0h335gcjen2pp4gjj2s2ax92x+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party2'
       "--out-file", "/dev/null",            # '/dev/null'
       "--submit", str(600)]                 # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17"




In [338]:
# save the transaction id
TX1_IN = result.stdout.split('"')[1]
TX1_IN

'4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17'

#### Check that the funds are distributed correctly
Notice that both wallets now have a common TxHash value.  

In [520]:
# check wallet1 and wallet2
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN


In [340]:
TX1_PARTY1_ADA    = TX1_IN+"#0"
TX1_PARTY1_TOKENS = TX1_IN+"#1"
TX1_PARTY2_ADA    = "ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33"+"#3"
TX1_PARTY2_TOKENS = TX1_IN+"#2"

## Deployment

### initialize (step1)
Generate the *.marlowe file with json files *.contract, *.state, Plutus data and network info.  
Question: (not sure here) The wallet addresses of the smart contract are not yet created on the blockchain, but known now.  

```text
Initialize the first transaction of a Marlowe contract and write output to a JSON file *.marlowe

marlowe-cli run initialize:

  --testnet-magic INTEGER          Network magic
  --socket-path SOCKET_FILE        Location of the cardano-node socket file
  --stake-address ADDRESS          Stake address
  --roles-currency CURRENCY_SYMBOL The currency symbol for roles
  --contract-file CONTRACT_FILE    JSON input file for the contract
  --state-file STATE_FILE          JSON input file for the contract state
  --out-file OUTPUT_FILE           JSON output file for initialize
  --print-stats                    Print statistics
```

In [343]:
cmd = [marloweCLI, "run", "initialize",     # Initialize the first transaction of a Marlowe contract
       "--testnet-magic", MAGIC,            # '1567'
       "--socket-path", SOCKET,             # '/tmp/node.socket'
       "--roles-currency", TOKEN_POLICY_ID, # '0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43'
       "--contract-file", "tx-1.contract",  # 'tx-1.contract'
       "--state-file", "tx-1.state",        # 'tx-1.state'
       "--out-file", "tx-1.marlowe",        # 'tx-1.marlowe'
       "--print-stats"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)



Validator size: 12415
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24652144, exBudgetMemory = ExMemory 82900}



In [344]:
!date
!ls -l tx-1*

Thu Jul 21 19:21:32 CEST 2022
-rw-rw-r-- 1 ubuntu ubuntu  1738 Jul 21 17:40 tx-1.contract
-rw-rw-r-- 1 ubuntu ubuntu 34273 Jul 21 19:21 tx-1.marlowe
-rw-rw-r-- 1 ubuntu ubuntu  1049 Jul 12 12:09 tx-1.raw
-rw-rw-r-- 1 ubuntu ubuntu   166 Jul 21 17:40 tx-1.state


In [345]:
# show content of file 
!cat tx-1.marlowe

{
    "state": {
        "choices": [],
        "boundValues": [],
        "accounts": [
            [
                [
                    {
                        "role_token": "Party1"
                    },
                    {
                        "currency_symbol": "",
                        "token_name": ""
                    }
                ],
                3000000
            ]
        ],
        "minTime": 1
    },
    "payments": [],
    "contract": {
        "timeout": 1658454012000,
        "when": [
            {
                "then": {
                    "timeout": 1658454012000,
                    "when": [
                        {
                            "then": {
                                "then": {
                                    "then": "close",
                                    "to": {
                                        "party": {
                                            "role_token": "Party1"
                                

### initial deposit (step2)
In the state file it was hard-coded to pay the initial deposit of 3 ADA,  
in order for the contract to be activated/initialized.  
A transaction fee will also be taken.  

```text
Run a Marlowe transaction

marlowe-cli run execute:

  --testnet-magic INTEGER          Network magic
  --socket-path SOCKET_FILE        Location of the cardano-node socket file
  --marlowe-in-file MARLOWE_FILE   JSON file with the Marlowe initial state and initial contract
  --tx-in-marlowe TXID#TXIX        UTxO spent from Marlowe contract
  --tx-in-collateral TXID#TXIX     Collateral for transaction
  --marlowe-out-file MARLOWE_FILE  JSON file with the Marlowe inputs, final state, and final contract
  --tx-in TXID#TXIX                Transaction input in TxId#TxIx format
  --tx-out ADDRESS+VALUE           Transaction output in ADDRESS+VALUE format
  --change-address ADDRESS         Address to receive ADA in excess of fee
  --required-signer SIGNING_FILE   File containing a required signing key
  --metadata-file METADATA_FILE    JSON file containing metadata
  --out-file FILE                  Output file for transaction body
  --submit SECONDS                 Also submit the transaction, and wait for confirmation
  --print-stats                    Print statistics
  --script-invalid                 Assert that the transaction is invalid
```

In [346]:
# Run a marlowe contract on the blockchain 'run execute'
cmd = [marloweCLI, "run", "execute",          # Run a Marlowe transaction
       "--testnet-magic", MAGIC,              # '1567'
       "--socket-path", SOCKET,               # '/tmp/node.socket'
       "--tx-in", TX1_PARTY1_ADA,             # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#0'
       "--change-address", PARTY1_ADDRESS,    # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--required-signer", PARTY1_SKEYFILE,  # 'wallet1.skey'
       "--marlowe-out-file", "tx-1.marlowe",  # 'tx-1.marlowe' JSON file with the Marlowe inputs, final state, and final contract
       "--out-file", "tx-1.raw",              # 'tx-1.raw'
       "--print-stats",
       "--submit", str(600)]                  # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

TxId "7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb"


Fee: Lovelace 187545
Size: 488 / 32768 = 1%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%



In [347]:
# save the transaction id
TX2_IN = result.stdout.split('"')[1]
TX2_IN

'7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb'

### prepare contract (stage1-2) 

```text
Prepare the next step of a Marlowe contract and write the output to a JSON file

marlowe-cli run prepare:

  --marlowe-file MARLOWE_FILE     JSON input file for the Marlowe state and contract
  --deposit-account PARTY         The account for the deposit
  --deposit-party PARTY           The party making the deposit
  --deposit-token TOKEN           The token being deposited, if not ADA
  --deposit-amount INTEGER        The amount of token being deposited
  --choice-name NAME              The name of the choice made
  --choice-party PARTY            The party making the choice
  --choice-number INTEGER         The number chosen
  --notify                        Notify the contract
  --invalid-before POSIX_TIME     Minimum time for the input, in POSIX milliseconds
  --invalid-hereafter POSIX_TIME  Maximum time for the input, in POSIX milliseconds
  --out-file OUTPUT_FILE          JSON output file for contract
  --print-stats                   Print statistics
```

In [352]:
# I wonder about the NOW+9*HOUR is nowhere written in the contract itself,
# we need to further make a drawing explaining and testing what the effects and menings are of these deadlines and why we need them...
# question: why is it not str(NOW+10*HOUR) ?
# question: this is the hardcoding of the next stage script, right ?
# question: is this invalid hereafter time something new, for let's say within this time the next wallet must respond or the contract ends ?

In [353]:
# Simulate the operation of a contract
cmd = [marloweCLI, "run", "prepare",                       # Prepare the next step
       "--marlowe-file", "tx-1.marlowe",                   # 'tx-1.marlowe'
       "--deposit-account", "Role="+PARTY1_TOKENASSETNAME, # 'Role=Party1'
       "--deposit-party", "Role="+PARTY1_TOKENASSETNAME,   # 'Role=Party1'
       "--deposit-amount", str(PARTY1_AMOUNT),             # '200000000'
       "--invalid-before", str(NOW),                       # '1658239769000'
       "--invalid-hereafter", str(NOW+9*HOUR),             # '1658272169000'
       "--out-file", "tx-2.marlowe",                       # 'tx-2.marlowe'
       "--print-stats"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)



Datum size: 202



### deposit 200 ADA (step3)

In [524]:
# get the contract address from the smart contract script file
marloweValidatorAddress = !jq -r .marloweValidator.address tx-1.marlowe

# save the address
CONTRACT_ADDRESS = marloweValidatorAddress[0]

# weblink to the explorer
print("https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address="+str(CONTRACT_ADDRESS))

# automatically open the weblink
display(Javascript('window.open("{url}");'.format(url="https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address="+str(marloweValidatorAddress[0]))))

https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address=addr_test1wqzctw6fur749cqp8ae9y8e6l6g9uvs9t4pv7thp0zy38cssc9g3z


<IPython.core.display.Javascript object>

In [527]:
# get the redeem address from the smart contract script file
marloweValidatorAddress = !jq -r '.rolesValidator.address' tx-1.marlowe

# save the address
REDEEM_ADDRESS = marloweValidatorAddress[0]

# weblink to the explorer
print("https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address="+str(REDEEM_ADDRESS))

https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address=addr_test1wqzuqf8hvl9d47d6kftykktrgszgu3yznrrpdgha9naxcmgr8le36


In [528]:
# weblink to the explorer for wallet 1
print("https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address="+str(PARTY1_ADDRESS))

https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address=addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t


In [529]:
# weblink to the explorer for wallet 2
print("https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address="+str(PARTY1_ADDRESS))

https://explorer.pioneers.testnet.marlowe-finance.io/en/address?address=addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t


In [530]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
2,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
3,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN
4,addr_test1wqzctw6fur749cqp8ae9y8e6l6g9uvs9t4pv7thp0zy38cssc9g3z,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,1,3000000,NaN,NaN,NaN,0,0e810f91611be5f0023233e5aa189dd099ad96d05437ffe0bc59a0f9a0137d64
5,addr_test1wqzuqf8hvl9d47d6kftykktrgszgu3yznrrpdgha9naxcmgr8le36,NaN,0,0,NaN,NaN,NaN,0,NaN


In [ ]:
TX2_CONTRACT     = TX2_IN + "#1"
TX2_PARTY1_ADA   = TX2_IN + "#0"
TX2_PARTY1_TOKEN = TX1_IN + "#1"
TX2_PARTY2_ADA   = "09c3f748b27145d998b392058269cb4f8716f4ec6c5f97deeef6ad5ae409d44b" + "#3"
TX2_PARTY2_TOKEN = TX1_IN + "#2"

In [ ]:
cmd = [marloweCLI, "run", "execute",         # Run a Marlowe transaction
       "--testnet-magic", MAGIC,             # '1567'
       "--socket-path", SOCKET,              # '/tmp/node.socket'
       "--marlowe-in-file", "tx-1.marlowe",  # 'tx-1.marlowe'
       "--tx-in-marlowe", TX2_CONTRACT,      # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373#1'
       "--tx-in-collateral", TX2_PARTY1_ADA, # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373#0'
       "--tx-in", TX2_PARTY1_ADA,            # 'f773c71f9b58dd95fd470aedd9df800838e841aef807102abcd87e3c881fd373#0'
       "--tx-in", TX2_PARTY1_TOKEN,          # '5906e4431fbd83cb9d16862ac7a4278ce707061af1d934f41f00abe2f7f150e9#1'
       "--change-address", PARTY1_ADDRESS,   # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t'
       "--required-signer", PARTY1_SKEYFILE, # 'wallet1.skey'
       "--marlowe-out-file", "tx-2.marlowe", # 'tx-2.marlowe'
       "--tx-out", PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,
       # 'addr_test1vppqe9wxg73zfll8yj2v3xg44k8pgnl0k4899pv3r3r2h2qlgsg4t+3000000+1 0b8ffef9a24d8dcdcd7618d74df4753aff6b6b7787b88ac69b4d7e43.Party1'
       "--out-file", "tx-2.raw",             # 'tx-2.raw'
       "--print-stats",
       "--submit", "600"]                    # '600'
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [ ]:
TX3_IN = result.stdout.split('"')[1]
TX3_IN

In [ ]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

In [ ]:
TX3_CONTRACT     = TX3_IN + "#1"
TX3_PARTY1_ADA   = TX3_IN + "#0"
TX3_PARTY1_TOKEN = TX3_IN + "#2"

### prepare contract (stage2-3)

In [ ]:
cmd = [marloweCLI, "run", "prepare",                       # Prepare the next step
       "--marlowe-file", "tx-2.marlowe",                   # 'tx-2.marlowe'
       "--deposit-account", "Role="+PARTY2_TOKENASSETNAME, # 'Party2'
       "--deposit-party", "Role="+PARTY2_TOKENASSETNAME,   # 'Party2'
       "--deposit-amount", str(PARTY2_AMOUNT),             # '100000000'
       "--invalid-before", str(NOW),                       # '1658239769000'
       "--invalid-hereafter", str(NOW+9*HOUR),             # '1658272169000'
       "--out-file", "tx-3.marlowe",                       # 'tx-3.marlowe'
       "--print-stats"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

### deposit 100 ADA (step4)

In [ ]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

In [ ]:
# Run a marlowe contract on the blockchain 'run execute'
cmd = [marloweCLI, "run", "execute", 
       "--testnet-magic", MAGIC,
       "--socket-path", SOCKET,
       "--marlowe-in-file", "tx-2.marlowe",
       "--tx-in-marlowe", TX3_CONTRACT,
       "--tx-in-collateral", TX2_PARTY2_ADA,
       "--tx-in", TX2_PARTY2_ADA,
       "--tx-in", TX2_PARTY2_TOKEN,
       "--change-address", PARTY2_ADDRESS,
       "--required-signer", PARTY2_SKEYFILE,
       "--marlowe-out-file", "tx-3.marlowe",
       "--tx-out", PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,
       "--out-file", "tx-3.raw",
       "--print-stats",
       "--submit", "600"]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [ ]:
TX4_IN = result.stdout.split('"')[1]
TX4_IN

In [ ]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

In [ ]:
TX_PARTY1_ADA   = TX3_IN + "#0"
TX_PARTY1_TOKEN = TX3_IN + "#2"
TX_PARTY2_ADA   = TX4_IN + "#0"
TX_PARTY2_TOKEN = TX4_IN + "#3"
print(TX_PARTY1_ADA, TX_PARTY1_ADA, TX_PARTY2_ADA, TX_PARTY2_TOKEN)

### withdraw (step5)

In [ ]:
# tx-in-collateral is the wallet with funds from the PARTY_1_ROLE just some ADA for fee available to bre used for allowing a transaction to occur
# tx-in is the original wallet with only ADA currency that needs to be consumed and replaced after a Transaction and is to allow for new funds to appear and change this storage of ADA
# tx-in (second) is to allow for using the token for that is needed as defined in the contract to give permission using the token to do a transaction...

In [ ]:
cmd = [marloweCLI, "run", "withdraw", 
       "--testnet-magic", MAGIC,
       "--socket-path", SOCKET,
       "--marlowe-file", "tx-3.marlowe",      # last contract version
       "--role-name", PARTY1_TOKENASSETNAME,
       "--tx-in-collateral", TX_PARTY1_ADA,
       "--tx-in", TX_PARTY1_ADA,
       "--tx-in", TX_PARTY1_TOKEN,
       "--change-address", PARTY1_ADDRESS,
       "--required-signer", PARTY1_SKEYFILE,
       "--tx-out", PARTY1_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY1_TOKENASSETNAME,   # needed against dos attack of 1000 calls
       "--out-file", "/dev/null",
       "--print-stats",
       "--submit", str(600)]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [ ]:
cmd = [marloweCLI, "run", "withdraw", 
       "--testnet-magic", MAGIC,
       "--socket-path", SOCKET,
       "--marlowe-file", "tx-3.marlowe",      # last contract version
       "--role-name", PARTY2_TOKENASSETNAME,
       "--tx-in-collateral", TX_PARTY2_ADA,
       "--tx-in", TX_PARTY2_ADA,
       "--tx-in", TX_PARTY2_TOKEN,
       "--change-address", PARTY2_ADDRESS,
       "--required-signer", PARTY2_SKEYFILE,
       "--tx-out", PARTY2_ADDRESS+"+"+str(MINIMUM_ADA)+"+"+str(1)+" "+TOKEN_POLICY_ID+"."+PARTY2_TOKENASSETNAME,   # needed against dos attack of 1000 calls
       "--out-file", "/dev/null",
       "--print-stats",
       "--submit", str(600)]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

In [ ]:
# check wallet1, wallet2 and contract, redeem address
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)]).reset_index(drop=True)

In [ ]:
# so now 100 ADA has been withdrawn or redeemed to the Party1
# also the 3 ADA used for the script activation is returned to the initiator of the contract.

### Function for Pandas Dataframe

In [ ]:
cardanoCliDataframe(PARTY_1_ADDRESS, MAGIC, 'full')

In [ ]:
cardanoCliDataframe(PARTY_1_ADDRESS, MAGIC)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", PARTY1_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", PARTY2_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", CONTRACT_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", REDEEM_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [ ]:
regex = re.compile(r'(\S{64})\s+(\d+)\s+(\d{6,})\s+(\S+)\s+[+]\s+(.+)')

In [ ]:
result.stdout

In [ ]:
df = pd.DataFrame(regex.findall(result.stdout), columns=['TxHash', 'TxIx', 'AmountLovelace', 'Currency', 'tail'])
df

In [ ]:
df.join(df['tail'].str.replace('.', ' ', regex=False).str.replace(' + ', ' ', regex=False).str.replace('"', '', regex=False).str.split(' ', regex=False, expand=True), how='left')

In [ ]:
df['tail'].iloc[2] = 'TxOutDatumHash ScriptDataInAlonzoEra "e13945ede9a5b560e25c175068ce0b955402ec7558090fbf510d7470d840e89f"'

In [ ]:
df

In [ ]:
df.join(df['tail'].str.replace('.', ' ', regex=False).str.replace(' + ', ' ', regex=False).str.replace('"', '', regex=False).str.split(' ', regex=False, expand=True), how='left')

In [ ]:
df = df.join(df['tail'].str.replace('.', ' ', regex=False).str.replace(' + ', ' ', regex=False).str.replace('"', '', regex=False).str.split(' ', regex=False, expand=True), how='left')
df

In [ ]:
df.columns = ['TxHash', 'TxIx', 'AmountLovelace', 'Currency', 'tail', 'TokenAmount', 'TokenPolicyId', 'TokenAssetNameHex', 'TxOut']
df

In [ ]:
df = df.drop(['tail', 'TxOut'], axis=1)
df

In [ ]:
df = df.replace({'TokenAmount':{'TxOutDatumNone': 0}})

In [ ]:
def hex_to_ascii(s):
    try:
        return bytearray.fromhex(s).decode()
    except ValueError:
        return None

In [ ]:
df['TokenAssetNameAscii'] = df['TokenAssetNameHex'].astype('str').apply(hex_to_ascii)

In [ ]:
df['TxHashSmall'] = df['TxHash'].str[-4::1] # get last 4 characters

In [ ]:
df['TokenPolicyIdSmall'] = df['TokenPolicyId'].str[-4::1] # get last 4 characters

In [ ]:
df['AmountAda'] = df['AmountLovelace'].astype(float)/1000000

In [ ]:
df[['TxHashSmall', 'TxIx', 'AmountAda', 'TokenAmount', 'TokenPolicyIdSmall']]

In [ ]:
df

In [ ]:
'TokenPolicyId', 'TokenAssetNameHex', 'TokenAssetNameAscii'

In [513]:
pd.concat([query_df(PARTY1_ADDRESS), query_df(PARTY2_ADDRESS), query_df(CONTRACT_ADDRESS), query_df(REDEEM_ADDRESS)])

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenNameHex,TokenNameAscii,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,None,0,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,Party1,1,NaN
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,Party2,1,NaN
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,None,0,NaN
0,addr_test1wqzctw6fur749cqp8ae9y8e6l6g9uvs9t4pv7thp0zy38cssc9g3z,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,1,3000000,NaN,NaN,NaN,0,0e810f91611be5f0023233e5aa189dd099ad96d05437ffe0bc59a0f9a0137d64
0,addr_test1wqzuqf8hvl9d47d6kftykktrgszgu3yznrrpdgha9naxcmgr8le36,NaN,0,0,NaN,NaN,NaN,0,NaN


In [489]:
query_df(PARTY1_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,0,490451577,NaN,NaN,NaN,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,1,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747931,1.0,NaN


In [490]:
query_df(PARTY2_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,1.0,NaN
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN,NaN


In [491]:
query_df(CONTRACT_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1wqzctw6fur749cqp8ae9y8e6l6g9uvs9t4pv7thp0zy38cssc9g3z,7cbef19b7c372a9b2ac2bc376f5564ffb84f291230c92c881a0083f18705dadb,1,3000000,NaN,NaN,NaN,0e810f91611be5f0023233e5aa189dd099ad96d05437ffe0bc59a0f9a0137d64


In [492]:
query_df(REDEEM_ADDRESS)

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount,datumhash
0,addr_test1wqzuqf8hvl9d47d6kftykktrgszgu3yznrrpdgha9naxcmgr8le36,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# interesting resource
# https://github.com/input-output-hk/marlowe-cardano/blob/c91f711b5132620dae7713dcc5a8607605b38f8f/marlowe-cli/cookbook/english-auction.ipynb

In [476]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", REDEEM_ADDRESS]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------



In [477]:
# query the current balance:
cmd = [cardanoCLI, "query", "utxo", "--testnet-magic", MAGIC, "--address", REDEEM_ADDRESS, "--out-file", 'party1_address.json']
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

In [478]:
!cat party1_address.json

{}

In [479]:
df = pd.read_json('party1_address.json', orient='index').reset_index()
df

,index


In [469]:
# replace feature names with interpretable naming (others are left as-is) 
df = df.rename(columns={'index': 'TxHash', 'datumhash': 'datumhash', 'address': 'address', 'value': 'value'})
df

,TxHash


In [470]:
# df.columns = ['TxHash', 'address', 'value']
# df

In [471]:
df = df.join(df['TxHash'].str.split('#', regex=False, expand=True), how='left')
df

AttributeError: Can only use .str accessor with string values!

In [445]:
df = df.drop(['TxHash'], axis=1)
df

,address,value,0,1
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508': {'506172747932'...,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'lovelace': 500000000},ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3


In [446]:
# replace feature names with interpretable naming (others are left as-is) 
df = df.rename(columns={0: 'TxHash', 1: 'TxIx'})
df

,address,value,TxHash,TxIx
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508': {'506172747932'...,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,{'lovelace': 500000000},ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3


In [447]:
# df.columns = ['address', 'value', 'TxHash', 'TxIx']
# df

In [448]:
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000.0


In [449]:
df['lovelace'] = df['lovelace'].astype('int')
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000


In [450]:
df['TxIx'] = df['TxIx'].astype('int')
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000


In [451]:
# drop all columns with all values NA
df = df.dropna(axis=1, how='all')
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000


In [452]:
# if the last column name word size is 56 characters wide:
# cn = df.iloc[:, -1].name
try:
    cn = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace'])]].iloc[:, -1].name
    if len(cn) == 56:
        print("yes, len(cn) == 56")
        # make a copy
        df['TokenPolicyId'] = df[cn]
        # write the column name only on True location
        df['TokenPolicyId'] = df['TokenPolicyId'].notna().replace({True: cn, False: np.nan})
except:
    print("except happened")
    df['TokenPolicyId'] = np.nan
    df['TokenAssetNameHex'] = np.nan
    df['TokenAmount'] = np.nan
    pass

yes, len(cn) == 56


In [453]:
df

,address,TxHash,TxIx,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,lovelace,TokenPolicyId
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,{'506172747932': 1},3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,NaN,500000000,NaN


In [454]:
df = pd.concat([df.drop([cn], axis=1), df[cn].apply(pd.Series)], axis=1)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932,0
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0,NaN
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN


In [455]:
# drop all columns with all values NA
df = df.dropna(axis=1, how='all')
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN


In [456]:
# cn = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]].iloc[:, -1].name
# # make a copy 
# df['TokenAssetNameHex'] = df[cn]
# # write the column name only on True location
# df['TokenAssetNameHex'] = df['TokenAssetNameHex'].notna().replace({True: cn, False: np.nan})

In [457]:
two = df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]]
two

,506172747932
0,1.0
1,NaN


In [458]:
tmp = two.stack().apply(pd.Series).reset_index()
tmp

,level_0,level_1,0
0,0,506172747932,1.0


In [459]:
tmp = tmp.set_index('level_0')
tmp

,level_1,0
level_0,,
0,506172747932,1.0


In [460]:
tmp.columns = ['TokenAssetNameHex', 'TokenAmount']
tmp

,TokenAssetNameHex,TokenAmount
level_0,,
0,506172747932,1.0


In [461]:
df = pd.concat([df, tmp], axis=1)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932,TokenAssetNameHex,TokenAmount
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0,506172747932,1.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN,NaN


In [462]:
# df['TokenAmount'] = df['TokenAmount'].astype('int')
# df

In [463]:
df = df.drop(tmp['TokenAssetNameHex'], axis=1)
df

,address,TxHash,TxIx,lovelace,TokenPolicyId,TokenAssetNameHex,TokenAmount
0,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,4c7f8ff0c3da6143147f2da4a3fccd30dee8dc4b71c0a74ee272557a1ec2ea17,2,3000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,506172747932,1.0
1,addr_test1vzn7ef2cx8h5y4jvxnn34pn3lzak5t537fx09j7zxdljs3gc5zgw6,ed62a729c96269053eef4aa5e4fabf9f6f85b7bb56f49481142ffcbe5f72aa33,3,500000000,NaN,NaN,NaN


In [ ]:
# df[df.columns[~df.columns.isin(['address', 'TxHash', 'TxIx', 'lovelace', 'TokenPolicyId'])]].stack()
# # make a copy 
# df['TokenAssetNameHex'] = df[cn]
# # write the column name only on True location
# df['TokenAssetNameHex'] = df['TokenAssetNameHex'].notna().replace({True: cn, False: np.nan})

In [85]:
# df

,address,TxHash,TxIx,lovelace,TokenPolicyId,506172747932,506172747931,TokenAssetNameHex
0,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea,0,479823763,NaN,NaN,NaN,NaN
1,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea,2,10000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,1.0,NaN,NaN
2,addr_test1vpha99j2s5slc5q26d884rhc3jzuxqex5javsak5enlg8wsel4z2t,e595b9fc55e0df7f8e0b8522628b8150e37cf249b4265e651e089cbf150ef3ea,1,10000000,6f4f37e5c163a2f77935f395434c49bc4de8d1894a3bd80ac239a508,NaN,1.0,506172747931


In [ ]:
# df = df.rename(columns={cn: 'TokenAmount'})
# df